# 基于CBOW实现Word2Vec

In [2]:
import io
import os
import sys
import requests
from collections import OrderedDict
import math
import random
import numpy as np
import paddle

## 1.数据处理

In [4]:
# 下载语料用于训练 word2vec
def download():
    corpus_url = "https://dataset.bj.bcebos.com/word2vec/text8.txt"
    web_request = requests.get(corpus_url)
    corpus = web_request.content
    with open("./text8.txt", "wb") as f:
        f.write(corpus)


download()

In [7]:
# 读取 text8 数据
def load_text8():
    with open("./text8.txt", "r") as f:
        corpus = f.read().strip("\n")
    f.close()

    return corpus


corpus = load_text8()

# 打印前 500 字符
print(corpus[:500])

 anarchism originated as a term of abuse first used against early working class radicals including the diggers of the english revolution and the sans culottes of the french revolution whilst the term is still used in a pejorative way to describe any act that used violent means to destroy the organization of society it has also been taken up as a positive label by self defined anarchists the word anarchism is derived from the greek without archons ruler chief king anarchism as a political philoso


In [8]:
# 对语料进行预处理操作
def data_preprocess(corpus):
    # 所有英文转为小写，归一化处理
    corpus = corpus.strip().lower()
    corpus = corpus.split(" ")

    return corpus


corpus = data_preprocess(corpus)
print(corpus[:50])

['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english', 'revolution', 'and', 'the', 'sans', 'culottes', 'of', 'the', 'french', 'revolution', 'whilst', 'the', 'term', 'is', 'still', 'used', 'in', 'a', 'pejorative', 'way', 'to', 'describe', 'any', 'act', 'that', 'used', 'violent', 'means', 'to', 'destroy', 'the']


In [9]:
# 构建词典
def build_dict(corpus):
    # 统计每个不同词频率
    word_freq_dict = dict()
    for word in corpus:
        if word not in word_freq_dict:
            word_freq_dict[word] = 0
        word_freq_dict[word] += 1

    # 将这个词典中的词，按照出现次数排序，出现次数越高，排序越靠前
    # 一般来说，出现频率高的高频词往往是：I，the，you这种代词，而出现频率低的词，往往是一些名词，如：nlp
    word_freq_dict = sorted(word_freq_dict.items(), key=lambda x: x[1], reverse=True)

    # 构造3个不同的词典，分别存储，
    # 每个词到id的映射关系：word2id_dict
    # 每个id出现的频率：word2id_freq
    # 每个id到词典映射关系：id2word_dict
    word2id_dict = dict()
    word2id_freq = dict()
    id2word_dict = dict()

    # 按照频率，从高到低，开始遍历每个单词，并为这个单词构造一个独一无二的id
    for word, freq in word_freq_dict:
        curr_id = len(word2id_dict)
        word2id_dict[word] = curr_id
        word2id_freq[word2id_dict[word]] = freq
        id2word_dict[curr_id] = word

    return word2id_freq, word2id_dict, id2word_dict


word2id_freq, word2id_dict, id2word_dict = build_dict(corpus)
vocab_size = len(word2id_freq)
print("there are totoally %d different words in the corpus" % vocab_size)
for _, (word, word_id) in zip(range(50), word2id_dict.items()):
    print("word %s, its id %d, its word freq %d" % (word, word_id, word2id_freq[word_id]))

there are totoally 253854 different words in the corpus
word the, its id 0, its word freq 1061396
word of, its id 1, its word freq 593677
word and, its id 2, its word freq 416629
word one, its id 3, its word freq 411764
word in, its id 4, its word freq 372201
word a, its id 5, its word freq 325873
word to, its id 6, its word freq 316376
word zero, its id 7, its word freq 264975
word nine, its id 8, its word freq 250430
word two, its id 9, its word freq 192644
word is, its id 10, its word freq 183153
word as, its id 11, its word freq 131815
word eight, its id 12, its word freq 125285
word for, its id 13, its word freq 118445
word s, its id 14, its word freq 116710
word five, its id 15, its word freq 115789
word three, its id 16, its word freq 114775
word was, its id 17, its word freq 112807
word by, its id 18, its word freq 111831
word that, its id 19, its word freq 109510
word four, its id 20, its word freq 108182
word six, its id 21, its word freq 102145
word seven, its id 22, its wor

根据 word2id 词典，把每个词转换成对应 id 序列

In [10]:
# 把语料转化成 id
def convert_corpus_to_id(corpus, word2id_dir):
    corpus = [word2id_dir[word] for word in corpus]
    return corpus


corpus = convert_corpus_to_id(corpus, word2id_dict)
print(corpus[:50])

[5233, 3080, 11, 5, 194, 1, 3133, 45, 58, 155, 127, 741, 476, 10571, 133, 0, 27349, 1, 0, 102, 854, 2, 0, 15067, 58112, 1, 0, 150, 854, 3580, 0, 194, 10, 190, 58, 4, 5, 10712, 214, 6, 1324, 104, 454, 19, 58, 2731, 362, 6, 3672, 0]


In [11]:
# 二次采样法处理样本
def subsampling(corpus, word2id_freq):
    def discard(word_id):
        return random.uniform(0, 1) < 1 - math.sqrt(1e4 / word2id_freq[word_id] * len(corpus))

    corpus = [word for word in corpus if not discard(word)]
    return corpus


corpus = subsampling(corpus, word2id_freq)
print(corpus[:50])

[5233, 3080, 11, 5, 194, 1, 3133, 45, 58, 155, 127, 741, 476, 10571, 133, 0, 27349, 1, 0, 102, 854, 2, 0, 15067, 58112, 1, 0, 150, 854, 3580, 0, 194, 10, 190, 58, 4, 5, 10712, 214, 6, 1324, 104, 454, 19, 58, 2731, 362, 6, 3672, 0]


In [12]:
# 构造数据，准备模型训练
# max_window_size 代表了最大的window_size的大小，程序会根据max_window_size从左到右扫描整个语料
# negative_sample_num 代表了对于每个正样本，我们需要随机采样多少负样本用于训练，
# 一般来说，negative_sample_num的值越大，训练效果越稳定，但是训练速度越慢。
def build_data(corpus, word2id_dict, word2id_freq, max_window_size=3, negative_sample_num=4):
    # 使用一个 list 存储处理好的数据
    dataset = []
    center_word_idx = 0

    # 从左到右，枚举每个中心点
    while center_word_idx < len(corpus):
        # 以 max_window_size 为上限，随机采样一个 window_size
        window_size = random.randint(1, max_window_size)
        # 当前中心词就是 center_word_idx 指向的词，可用作正样本
        positive_word = corpus[center_word_idx]

        # 以当前中心词为中心，左右两侧在 window_size 内的词就是上下文
        context_word_range = (
            max(0, center_word_idx - window_size), min(len(corpus) - 1, center_word_idx + window_size))
        # 上下文词
        context_word_candidates = [corpus[idx] for idx in range(context_word_range[0], context_word_range[1] + 1) if
                                   idx != center_word_idx]

        # 对于正样本来说，随机采样 negative_sample_num 个负样本用于训练
        for context_word in context_word_candidates:
            # 首先把 (上下文, 正样本, label=1) 放在三元组数组 dataset 中
            # label = 1 表示正样本
            dataset.append((context_word, positive_word, 1))

            # 开始负采样
            i = 0
            while i < negative_sample_num:
                negative_word_candidate = random.randint(0, vocab_size - 1)

                if negative_word_candidate is not positive_word:
                    # 首先把 (上下文, 负样本, label=1) 放在三元组数组 dataset 中
                    # label = 0 表示负样本
                    dataset.append((context_word, negative_word_candidate, 0))
                    i += 1

        center_word_idx = min(len(corpus) - 1, center_word_idx + window_size)
        if center_word_idx == (len(corpus) - 1):
            center_word_idx += 1
        if center_word_idx % 100000 == 0:
            print(center_word_idx)

    return dataset


dataset = build_data(corpus, word2id_dict, word2id_freq)
for _, (context_word, target_word, label) in zip(range(50), dataset):
    print("center_word %s, target %s, label %d" % (id2word_dict[context_word],
                                                   id2word_dict[target_word], label))

300000
600000
700000
800000
1300000
1400000
1800000
1900000
2000000
2200000
2300000
2500000
2700000
2800000
3300000
3500000
3700000
3800000
4100000
4200000
4300000
4600000
5000000
5100000
5200000
5300000
5500000
5700000
5800000
6000000
6100000
6200000
6300000
6700000
6800000
7000000
7300000
8100000
8300000
8500000
8700000
8800000
8900000
9100000
9500000
9800000
10000000
10100000
10200000
10400000
10500000
10900000
11500000
11800000
12100000
12200000
12300000
12500000
12600000
13000000
13200000
13300000
13500000
13600000
13800000
13900000
14300000
14400000
14800000
15000000
15100000
15200000
15300000
15500000
15600000
16000000
16100000
16200000
16300000
16500000
16700000
16900000
center_word originated, target anarchism, label 1
center_word originated, target barites, label 0
center_word originated, target attillement, label 0
center_word originated, target negotiative, label 0
center_word originated, target excavator, label 0
center_word anarchism, target originated, label 1
center_wor

In [13]:
#构造mini-batch，准备对模型进行训练
#我们将不同类型的数据放到不同的tensor里，便于神经网络进行处理
#并通过numpy的array函数，构造出不同的tensor来，并把这些tensor送入神经网络中进行训练
def build_batch(dataset, batch_size, epoch_num):
    #center_word_batch缓存batch_size个中心词
    center_word_batch = []
    #target_word_batch缓存batch_size个目标词（可以是正样本或者负样本）
    target_word_batch = []
    #label_batch缓存了batch_size个0或1的标签，用于模型训练
    label_batch = []

    for epoch in range(epoch_num):
        #每次开启一个新epoch之前，都对数据进行一次随机打乱，提高训练效果
        random.shuffle(dataset)

        for center_word, target_word, label in dataset:
            #遍历dataset中的每个样本，并将这些数据送到不同的tensor里
            center_word_batch.append([center_word])
            target_word_batch.append([target_word])
            label_batch.append(label)

            #当样本积攒到一个batch_size后，我们把数据都返回回来
            #在这里我们使用numpy的array函数把list封装成tensor
            #并使用python的迭代器机制，将数据yield出来
            #使用迭代器的好处是可以节省内存
            if len(center_word_batch) == batch_size:
                yield np.array(center_word_batch).astype("int64"), \
                    np.array(target_word_batch).astype("int64"), \
                    np.array(label_batch).astype("float32")
                center_word_batch = []
                target_word_batch = []
                label_batch = []

    if len(center_word_batch) > 0:
        yield np.array(center_word_batch).astype("int64"), \
            np.array(target_word_batch).astype("int64"), \
            np.array(label_batch).astype("float32")

## 网络定义

In [ ]:

#定义cbow训练网络结构
#这里我们使用的是paddlepaddle的2.0.0版本
#一般来说，在使用nn训练的时候，我们需要通过一个类来定义网络结构，这个类继承了paddle.nn.Layer
class CBOW(paddle.nn.Layer):
    def __init__(self, vocab_size, embedding_size, init_scale=0.1):
        #vocab_size 定义了这个CBOW这个模型的词表大小
        #embedding_size 定义了词向量的维度是多少
        #init_scale 定义了词向量初始化的范围，一般来说，比较小的初始化范围有助于模型训练
        super(CBOW, self).__init__()
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size

        #使用 paddle.nn 提供的 Embedding 函数，构造一个词向量参数
        #这个参数的大小为：self.vocab_size, self.embedding_size
        #这个参数的名称为：embedding_para
        #这个参数的初始化方式为在[-init_scale, init_scale]区间进行均匀采样
        self.embedding = paddle.nn.Embedding(
            self.vocab_size,
            self.embedding_size,
            weight_attr=paddle.ParamAttr(
                name='embedding_para',
                initializer=paddle.nn.initializer.Uniform(
                    low=-0.5 / embedding_size, high=0.5 / embedding_size)))

        #使用paddle.nn提供的Embedding函数，构造另外一个词向量参数
        #这个参数的大小为：self.vocab_size, self.embedding_size
        #这个参数的名称为：embedding_para_out
        #这个参数的初始化方式为在[-init_scale, init_scale]区间进行均匀采样
        #跟上面不同的是，这个参数的名称跟上面不同，因此，
        #embedding_para_out和embedding_para虽然有相同的shape，但是权重不共享
        self.embedding_out = paddle.nn.Embedding(
            self.vocab_size,
            self.embedding_size,
            weight_attr=paddle.ParamAttr(
                name='embedding_out_para',
                initializer=paddle.nn.initializer.Uniform(
                    low=-0.5 / embedding_size, high=0.5 / embedding_size)))

    #定义网络的前向计算逻辑
    #center_words是一个tensor（mini-batch），表示中心词
    #target_words是一个tensor（mini-batch），表示目标词
    #label是一个tensor（mini-batch），表示这个词是正样本还是负样本（用0或1表示）
    #用于在训练中计算这个tensor中对应词的同义词，用于观察模型的训练效果
    def forward(self, center_words, target_words, label):
        #首先，通过embedding_para（self.embedding）参数，将mini-batch中的词转换为词向量
        #这里center_words和eval_words_emb查询的是一个相同的参数
        #而target_words_emb查询的是另一个参数
        center_words_emb = self.embedding(center_words)
        target_words_emb = self.embedding_out(target_words)

        #center_words_emb = [batch_size, embedding_size]
        #target_words_emb = [batch_size, embedding_size]
        #我们通过点乘的方式计算中心词到目标词的输出概率，并通过sigmoid函数估计这个词是正样本还是负样本的概率。
        word_sim = paddle.multiply(center_words_emb, target_words_emb)
        word_sim = paddle.sum(word_sim, axis=-1)
        word_sim = paddle.reshape(word_sim, shape=[-1])
        pred = paddle.nn.functional.sigmoid(word_sim)

        #通过估计的输出概率定义损失函数，注意我们使用的是binary_cross_entropy函数
        #将sigmoid计算和cross entropy合并成一步计算可以更好的优化，所以输入的是word_sim，而不是pred

        loss = paddle.nn.functional.binary_cross_entropy(paddle.nn.functional.sigmoid(word_sim), label)
        loss = paddle.mean(loss)

        #返回前向计算的结果，飞桨会通过backward函数自动计算出反向结果。
        return pred, loss

## 网络训练

In [ ]:
#开始训练，定义一些训练过程中需要使用的超参数
batch_size = 512
epoch_num = 3
embedding_size = 200
step = 0
learning_rate = 0.001

#定义一个使用word-embedding计算cos的函数
def get_cos(query1_token, query2_token, embed):
    W = embed
    x = W[word2id_dict[query1_token]]
    y = W[word2id_dict[query2_token]]
    cos = np.dot(x, y) / np.sqrt(np.sum(y * y) * np.sum(x * x) + 1e-9)
    flat = cos.flatten()
    print("单词1 %s 和单词2 %s 的cos结果为 %f" %(query1_token, query2_token, cos))


#通过我们定义的CBOW类，来构造一个cbow模型网络
skip_gram_model = CBOW(vocab_size, embedding_size)
#构造训练这个网络的优化器
adam = paddle.optimizer.Adam(learning_rate=learning_rate, parameters = skip_gram_model.parameters())

#使用build_batch函数，以mini-batch为单位，遍历训练数据，并训练网络
for center_words, target_words, label in build_batch(
    dataset, batch_size, epoch_num):
    #使用paddle.to_tensor函数，将一个numpy的tensor，转换为飞桨可计算的tensor
    center_words_var = paddle.to_tensor(center_words)
    target_words_var = paddle.to_tensor(target_words)
    label_var = paddle.to_tensor(label)

    #将转换后的tensor送入飞桨中，进行一次前向计算，并得到计算结果
    pred, loss = skip_gram_model(
        center_words_var, target_words_var, label_var)

    #通过backward函数，让程序自动完成反向计算
    loss.backward()
    #通过minimize函数，让程序根据loss，完成一步对参数的优化更新
    adam.minimize(loss)
    #使用clear_gradients函数清空模型中的梯度，以便于下一个mini-batch进行更新
    skip_gram_model.clear_gradients()

    #每经过100个mini-batch，打印一次当前的loss，看看loss是否在稳定下降
    step += 1
    if step % 100 == 0:
        print("step %d, loss %.3f" % (step, loss.numpy()[0]))

    #经过10000个mini-batch，打印一次模型对eval_words中的10个词计算的同义词
    #这里我们使用词和词之间的向量点积作为衡量相似度的方法
    #我们只打印了5个最相似的词
    if step % 2000 == 0:
        embedding_matrix = skip_gram_model.embedding.weight.numpy()
        np.save("./embedding", embedding_matrix)
        get_cos("king","queen",embedding_matrix)
        get_cos("she","her",embedding_matrix)
        get_cos("topic","theme",embedding_matrix)
        get_cos("woman","game",embedding_matrix)
        get_cos("one","name",embedding_matrix)

In [ ]:
#定义一个使用word-embedding计算cos的函数
def get_cos(query1_token, query2_token, embed):
    W = embed
    x = W[word2id_dict[query1_token]]
    y = W[word2id_dict[query2_token]]
    cos = np.dot(x, y) / np.sqrt(np.sum(y * y) * np.sum(x * x) + 1e-9)
    flat = cos.flatten()
    print("单词1 %s 和单词2 %s 的cos结果为 %f" %(query1_token, query2_token, cos) )

embedding_matrix = np.load('embedding.npy')
get_cos("king","queen",embedding_matrix)
get_cos("she","her",embedding_matrix)
get_cos("topic","theme",embedding_matrix)
get_cos("woman","game",embedding_matrix)
get_cos("one","name",embedding_matrix)